In [8]:
# ============================================
# 1. 모듈 임포트
# ============================================
from bs4 import BeautifulSoup
from collections import Counter
import pandas as pd
import re

# ============================================
# 2. 로컬 HTML 파일 읽기
# ============================================
HTML_PATH = "../Pr_Data/danawa_.html"  # 저장된 파일 경로

with open(HTML_PATH, "r", encoding="utf-8") as f:
    html = f.read()

soup = BeautifulSoup(html, "html.parser")

# ============================================
# 3. 제목 추출
# ============================================
title_tags = soup.select("td.contents div.title a")
titles = [t.get_text(strip=True) for t in title_tags]

# ============================================
# 4. 키워드 전처리
# ============================================
# 제목 전체를 하나의 문자열로 합치기
all_text = " ".join(titles)

# 특수문자 제거
clean_text = re.sub(r"[^0-9A-Za-z가-힣\s]", " ", all_text)

# 공백 기준 분리
tokens = clean_text.split()

# 불용어(필터링할 단어)
stopwords = ["영상", "기사", "리뷰", "오토헤럴드", "다나와자동차", "시승기"]

# 단어 길이 2 이상 & 불용어 제거
words = [w for w in tokens if len(w) >= 2 and w not in stopwords]

# ============================================
# 5. 단어 빈도 Top30 DataFrame
# ============================================
counter = Counter(words)
top30 = counter.most_common(30)

df_keywords = pd.DataFrame(top30, columns=["단어", "빈도"])

print("\n[키워드 Top 30]")
print(df_keywords)

# 필요하면 CSV 저장
# df_keywords.to_csv("danawa_top30.csv", index=False, encoding="utf-8-sig")



[키워드 Top 30]
       단어  빈도
0     전기차   3
1    일렉트릭   2
2     현대차   2
3      공개   2
4      기아   2
5     BYD   1
6      돌핀   1
7      인증   1
8      완료   1
9     캐스퍼   1
10     겨냥   1
11   초가성비   1
12     출격   1
13     대체   1
14     불가   1
15   플래그십   1
16    SUV   1
17  팰리세이드   1
18  하이브리드   1
19    반복적   1
20     가속   1
21    끝까지   1
22    버티는   1
23    포르쉐   1
24    카이엔   1
25  하이라이트   1
26    오르네   1
27   제네시스   1
28    고성능   1
29   GV60   1


In [16]:
# ============================================
# 1. Selenium 설정
# ============================================
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

# ChromeDriver 경로
driver = webdriver.Chrome()

titles = []   # ← 제목 누적 저장 리스트

page = 1

try:
    while True:
        print("\n" + "="*50)
        print(f"📌 [{page} 페이지 크롤링 시작]")
        print("="*50)

        url = f"https://auto.danawa.com/news/?Work=list&Tab=A&Stab=all&Brand=&Model=&HeadText=&NewsGroup=&SearchKey=&SearchWord=&Punit=10&useOldData=&Page={page}"
        driver.get(url)
        time.sleep(1.2)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # 제목 추출
        title_tags = soup.select("td.contents div.title a")
        if not title_tags:
            print("❗ 더 이상 데이터 없음 → 크롤링 종료")
            break

        # 이번 페이지에서 수집한 제목들
        page_titles = [t.get_text(strip=True) for t in title_tags]

        print(f"📄 {page} 페이지에서 추출된 제목 {len(page_titles)}개:")
        for idx, t in enumerate(page_titles, 1):
            print(f"   {idx}. {t}")

        # 누적 저장
        titles.extend(page_titles)

        print(f"\n🔄 현재까지 누적 제목 개수: {len(titles)}")

        page += 1
        time.sleep(1)

except KeyboardInterrupt:
    print("\n⛔ 사용자 중단 감지! 현재까지 수집한 데이터만 저장합니다.")

finally:
    driver.quit()

print("\n==================================================")
print(f"🎉 최종 누적 제목 개수: {len(titles)}")
print("==================================================")



📌 [1 페이지 크롤링 시작]
📄 1 페이지에서 추출된 제목 10개:
   1. 테슬라, 유럽 FSD 승인?… 네덜란드 차량관리청 “승인 절차 아니다” 반박
   2. ‘아이콘즈 오브 포르쉐’, 1세대 카이엔 존더분쉬와 카이엔 일렉트릭 공개
   3. 잔금 줄 테니 도면 달라, 빼돌린 기술, 경쟁사에 제공한 車부품 대기업
   4. '구리' 노린 전기차 충전 케이블 절단 범죄 기승... 급기야 이런 방법까지
   5. 포르쉐, ‘아이콘즈 오브 포르쉐’에서 1세대 카이엔 존더분쉬와 신형 카이엔 일렉트릭 공개
   6. 폭스바겐코리아, SBS ‘키스는 괜히 해서!’에 투아렉·ID.5·골프 GTI 협찬
   7. 한성자동차, 창립 40주년 맞아 ‘프리미엄 브랜드 경험’ 전략 강화
   8. 볼보자동차코리아, 광주 ‘볼보 셀렉트’ 전시장에 신규 고객 라운지 오픈
   9. 기아 EV6 GT, 독일 AMS 전기차 비교 평가 1위
   10. 아우디 트윈컵 2025, 한국 대표팀 세일즈·애프터 세일즈 모두 3위

🔄 현재까지 누적 제목 개수: 10

📌 [2 페이지 크롤링 시작]
📄 2 페이지에서 추출된 제목 10개:
   1. 현대차·기아, 미쉐린과 손잡았다… 전기차 고성능 타이어 공동 개발
   2. WRC 사우디아라비아 랠리 개막… 시즌 최종전, 오지에·에반스 챔피언십 향방 주목
   3. 보그워너, 장성자동차와 전동화 파워트레인 협력 강화
   4. 제네시스 마그마, 무엇을 추구하는가?
   5. 독일 자동차 산업 고용, 13년 만에 최저치 기록
   6. 위라이드, 2025년 3분기 적자 대폭 축소
   7. 샤오미, 올해 전기차 판매 목표 40만 대로 상향
   8. 테슬라, ‘감독 없는 FSD’ 테스트 지역 드러났다
   9. BYD ‘돌핀’ 인증 완료… 캐스퍼 일렉트릭 겨냥, 초가성비 전기차 출격
   10. '대체 불가' 플래그십 SUV 현대차 팰리세이드 하이브리드

🔄 현재까지 누적 제목 개수: 20

📌 [3 페이지 크롤링 시작]
📄 3 페이지에서 

WebDriverException: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=142.0.7444.175)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6adb2a235
	0x7ff6ad882630
	0x7ff6ad6116dd
	0x7ff6ad60e228
	0x7ff6ad5feba9
	0x7ff6ad6009d7
	0x7ff6ad5ff160
	0x7ff6ad5fe917
	0x7ff6ad5fe5db
	0x7ff6ad5fc145
	0x7ff6ad5fc9dc
	0x7ff6ad6159ea
	0x7ff6ad6bc8be
	0x7ff6ad692b0a
	0x7ff6ad6bbaba
	0x7ff6ad65b0ed
	0x7ff6ad65bf63
	0x7ff6adb55d60
	0x7ff6adb4fe8a
	0x7ff6adb71005
	0x7ff6ad89d71e
	0x7ff6ad8a4e1f
	0x7ff6ad88b7c4
	0x7ff6ad88b97f
	0x7ff6ad8718e8
	0x7ffdf0ede8d7
	0x7ffdf2d6c53c


In [ ]:
from collections import Counter
import pandas as pd
import re

# ============================================
# 1. 제목 리스트를 하나의 문자열로 합치기
# ============================================
all_text = " ".join(titles)

# 특수문자 제거
clean_text = re.sub(r"[^0-9A-Za-z가-힣\s]", " ", all_text)

# 공백 분리
tokens = clean_text.split()

# 불용어 (원하면 조정 가능)
stopwords = ["공개", "영상", "기사", "리뷰", "오토헤럴드", "다나와자동차", "시승기"]

# 길이 2 이상 단어만 필터링
words = [w for w in tokens if len(w) >= 2 and w not in stopwords]

# ============================================
# 2. 단어 빈도 Top30
# ============================================
counter = Counter(words)
top30 = counter.most_common(500)

df_keywords = pd.DataFrame(top30, columns=["단어", "빈도"])
df_keywords.to_csv('result.csv',index=False)
# cond = df_keywords['단어'] == '자율주행'

# df_new = df_keywords[cond]
# df_new